In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from skimage.feature import local_binary_pattern, hog
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time
from sklearn.decomposition import PCA
import joblib 

def is_dfire_image_fire(annotation_path, fire_class_ids):
    if not os.path.exists(annotation_path):
        return False
    try:
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts and len(parts) > 0:
                    if parts[0].isdigit():
                        class_id = int(parts[0])
                        if class_id in fire_class_ids:
                            return True
    except Exception as e:
        pass 
    return False

def extract_color_histograms(img_processed, color_space, bins):
    histograms = []
    if color_space == 'hsv':
        if img_processed.dtype == np.float32:
            hist_h = cv2.calcHist([img_processed], [0], None, [bins], [0, 180]) 
            hist_s = cv2.calcHist([img_processed], [1], None, [bins], [0, 1]) 
            histograms.extend([hist_h.flatten(), hist_s.flatten()])
        elif img_processed.dtype == np.uint8:
            hist_h = cv2.calcHist([img_processed], [0], None, [bins], [0, 180])
            hist_s = cv2.calcHist([img_processed], [1], None, [bins], [0, 256])
            histograms.extend([hist_h.flatten(), hist_s.flatten()])
    elif color_space == 'ycbcr':
        if img_processed.dtype == np.float32:
            hist_y = cv2.calcHist([img_processed], [0], None, [bins], [0, 1])
            hist_cb = cv2.calcHist([img_processed], [1], None, [bins], [-0.5, 0.5])
            hist_cr = cv2.calcHist([img_processed], [2], None, [bins], [-0.5, 0.5])
            histograms.extend([hist_y.flatten(), hist_cb.flatten(), hist_cr.flatten()])
        elif img_processed.dtype == np.uint8:
            hist_y = cv2.calcHist([img_processed], [0], None, [bins], [0, 256])
            hist_cb = cv2.calcHist([img_processed], [1], None, [bins], [0, 256])
            hist_cr = cv2.calcHist([img_processed], [2], None, [bins], [0, 256])
            histograms.extend([hist_y.flatten(), hist_cb.flatten(), hist_cr.flatten()])
    if histograms: return np.concatenate(histograms)
    else: return np.array([])

def extract_lbp_features(img_gray, radius, n_points, method):
    if n_points is None:
        n_points = 8 * radius
    lbp_image = local_binary_pattern(img_gray, n_points, radius, method=method)
    if method == 'uniform':
        n_bins = n_points + 2 
        hist_range = (0, n_points + 2)
    else:
        n_bins = int(lbp_image.max() + 1)
        hist_range = (0, n_bins)
        if lbp_image.size > 0: 
            n_bins = int(lbp_image.max() + 1)
        else:
            n_bins = 1 
        hist_range = (0, n_bins)
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=n_bins, range=hist_range)

    lbp_hist = lbp_hist.astype(np.float32)
    if lbp_hist.sum() > 0:
        lbp_hist /= lbp_hist.sum() 
    return lbp_hist.flatten()

def extract_hog_features(img_gray, orientations, pixels_per_cell, cells_per_block, block_norm):
    if img_gray.dtype == np.uint8:
         img_gray = img_gray.astype(np.float32) / 255.0 
    min_height = cells_per_block[0] * pixels_per_cell[0]
    min_width = cells_per_block[1] * pixels_per_cell[1]
    if img_gray.shape[0] < min_height or img_gray.shape[1] < min_width:
        return np.array([])

    try:
        hog_features = hog(img_gray, orientations=orientations,
                           pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block,
                           block_norm=block_norm,
                           visualize=False, feature_vector=True)
        return hog_features.flatten()
    except ValueError as e:
         return np.array([])

def combine_features(img_dict, feature_params):
    all_features = []
    if 'hsv' in img_dict:
        
        hsv_hist = extract_color_histograms(img_dict['hsv'], 'hsv', bins=feature_params.get('hist_bins', 100))
        if hsv_hist.size > 0:
            all_features.append(hsv_hist)
    if 'ycbcr' in img_dict:
         
         ycbcr_hist = extract_color_histograms(img_dict['ycbcr'], 'ycbcr', bins=feature_params.get('hist_bins', 100))
         if ycbcr_hist.size > 0:
              all_features.append(ycbcr_hist)

    if 'gray' in img_dict:
        img_gray_processed = img_dict['gray']

        lbp_features = extract_lbp_features(img_gray_processed,
                                            radius=feature_params.get('lbp_radius', 3),
                                            n_points=feature_params.get('lbp_n_points', None),
                                            method=feature_params.get('lbp_method', 'uniform'))
        if lbp_features.size > 0:
             all_features.append(lbp_features)

        hog_features = extract_hog_features(img_gray_processed,
                                           orientations=feature_params.get('hog_orientations', 9),
                                           pixels_per_cell=feature_params.get('hog_pixels_per_cell', (8, 8)),
                                           cells_per_block=feature_params.get('hog_cells_per_block', (2, 2)),
                                           block_norm=feature_params.get('hog_block_norm', 'L2-Hys'))
        if hog_features.size > 0:
             all_features.append(hog_features)

    if all_features:
        try:
            combined_vector = np.concatenate(all_features)
            return combined_vector
        except ValueError as e:
            return np.array([]) 

    else:
        return np.array([])


def get_config(dataset_choice):
    config = {}
    if dataset_choice == 'kaggle':
        config['data_root'] = "fire_dataset" 
        config['target_img_size'] = (128, 128) 
        config['color_spaces_to_load'] = ['bgr', 'hsv', 'ycbcr'] 
        config['normalize_pixels'] = 1 
        config['fire_class_ids'] = None 
    elif dataset_choice == 'dfire':
        config['dfire_root'] = "D-Fire"
        config['split_name'] = "test" 
        config['data_root'] = os.path.join(config['dfire_root'], config['split_name'])
        config['target_img_size'] = (128, 128) 
        config['color_spaces_to_load'] = ['bgr', 'hsv', 'ycbcr'] 
        config['normalize_pixels'] = 1 
        config['fire_class_ids'] = [0] 
    else:
        raise ValueError(f"Desteklenmeyen veri seti seçimi: {dataset_choice}")

    print(f"Kullanılan veri seti: {dataset_choice}")
    print(f"Veri kök dizini: {config.get('data_root')}")
    print(f"Hedef görüntü boyutu: {config['target_img_size']}")
    print(f"Yüklenecek renk uzayları: {config['color_spaces_to_load']}")
    print(f"Piksel normalizasyonu: {'Evet' if config['normalize_pixels'] else 'Hayır'}")
    if dataset_choice == 'dfire':
         print(f"D-Fire Bölmesi: {config['split_name']}")
         print(f"D-Fire Yangın Sınıfı ID'leri: {config['fire_class_ids']}")
    return config

def get_feature_params():
    feature_params = {
        'hist_bins': 100, 
        'lbp_radius': 3, 
        'lbp_n_points': None, 
        'lbp_method': 'uniform', 
        'hog_orientations': 9, 
        'hog_pixels_per_cell': (8, 8), 
        'hog_cells_per_block': (2, 2), 
        'hog_block_norm': 'L2-Hys' 
    }
    print("\nÖzellik çıkarma parametreleri:", feature_params)
    return feature_params

def load_and_preprocess_data(config):
    data_root = config['data_root']
    target_img_size = config['target_img_size']
    color_spaces_to_load = config['color_spaces_to_load']
    normalize_pixels = config['normalize_pixels']
    dataset_choice = config.get('dataset_choice', 'kaggle') 

    processed_data_dict = None
    if dataset_choice == 'kaggle':
         if os.path.exists(data_root):
            print(f"Kaggle veri seti yükleniyor: {data_root}")
            processed_data_dict = load_prep_img(
                data_root,
                target_img_size,
                color_spaces=color_spaces_to_load,
                normalize=normalize_pixels
            )
         else:
            print(f"Hata: Kaggle veri seti kök dizini bulunamadı: {data_root}")

    elif dataset_choice == 'dfire':
         fire_class_ids = config['fire_class_ids']
         if os.path.exists(data_root):
            print(f"D-Fire veri seti yükleniyor: {data_root}")
            processed_data_dict = load_prep_dfire(
                data_root,
                target_img_size,
                fire_class_ids=fire_class_ids,
                color_spaces=color_spaces_to_load,
                normalize=normalize_pixels
            )
         else:
            print(f"Hata: D-Fire bölmesi kök dizini bulunamadı: {data_root}")

    if processed_data_dict and 'labels' in processed_data_dict and len(processed_data_dict.get('labels', [])) > 0:
        print(f"\n{len(processed_data_dict['labels'])} görüntü başarıyla yüklendi ve ön işlendi.")
    else:
        print("Hata: Veri yüklenemedi veya ön işlenemedi.")
        return None 

    return processed_data_dict


def extract_features(processed_data_dict, feature_params):
    features = []
    valid_labels = []
    if processed_data_dict and 'labels' in processed_data_dict and len(processed_data_dict.get('labels', [])) > 0:
        labels_all = processed_data_dict['labels']
        num_images_total = len(labels_all)
        available_data_keys = [key for key in processed_data_dict.keys() if key != 'labels']
        print(f"Toplam {num_images_total} görüntüden özellikler çıkarılıyor.")
        for i in tqdm(range(num_images_total), desc=f"Özellikler Çıkarılıyor"):
            img_dict_single = {}
            for key in available_data_keys:
                 if processed_data_dict[key] is not None and i < len(processed_data_dict[key]):
                      img_dict_single[key] = processed_data_dict[key][i]
            if img_dict_single:
                features_single = combine_features(img_dict_single, feature_params)
                if features_single.size > 0:
                    features.append(features_single)
                    valid_labels.append(labels_all[i])
                else:pass 
            else:pass 

        if features:
            features_array = np.vstack(features).astype(np.float32)
            labels_array = np.array(valid_labels, dtype=np.int32)
            print(f"\nBaşarıyla özellik çıkarılan resim sayısı: {features_array.shape[0]}")
            print(f"Özellik vektörleri boyutu: {features_array.shape[1]}")
            print(f"Feature Array Shape: {features_array.shape}")
            print(f"Label Array Shape: {labels_array.shape}")
        else:
            print("\nHiçbir görüntüden özellik çıkarılamadı!")
            features_array = np.array([])
            labels_array = np.array([])
        return features_array, labels_array
    else:
        print("Hata: Özellik çıkarmak için ön işlenmiş veri yok veya boş.")
        return np.array([]), np.array([]) 


def split_data(features_array, labels_array, test_size=0.2, random_state=42):
    if features_array.shape[0] == 0:
        print("Özellik dizisi boş, veri bölünemedi.")
        return None, None, None, None

    print(f"\nVeri bölünüyor: Eğitim oranı ({1-test_size:.0%}), Test oranı ({test_size:.0%})")
    X_train, X_test, y_train, y_test = train_test_split(
        features_array,
        labels_array,
        test_size=test_size,
        random_state=random_state,
        stratify=labels_array 
    )
    print(f"Eğitim özellikleri boyutu: {X_train.shape}")
    print(f"Test özellikleri boyutu: {X_test.shape}")
    print(f"Eğitim etiketleri boyutu: {y_train.shape}")
    print(f"Test etiketleri boyutu: {y_test.shape}")
    return X_train, X_test, y_train, y_test

def scale_features_transform(X_data, scaler):
    if X_data is None or X_data.shape[0] == 0:
         print("scale edilemedi! data bos")
         return None
    try:
         X_scaled = scaler.transform(X_data)
         return X_scaled
    except Exception as e:
         print(f"Veri scale edilirken hata oluştu: {e}")
         return None

def apply_pca_transform(X_data, pca):
    if X_data is None or X_data.shape[0] == 0:
         return None
    try:
         X_pca = pca.transform(X_data)
         return X_pca
    except Exception as e:
         print(f"Veri PCA ile dönüştürülürken hata oluştu: {e}")
         return None

def load_prep_img(data_root, target_size,
    color_spaces=['bgr', 'hsv', 'ycbcr'], normalize=1):
    processed_data = {cs: [] for cs in color_spaces + ['gray']}
    labels = []
    class_dirs = {'fire_images': 1, 'non_fire_images': 0} 
    img_extensions = ['.jpg', '.jpeg', '.png']
    total_images_processed = 0
    total_images_skipped = 0
    print(f"Görüntüler yükleniyor ve ön işleniyor: {data_root}")
    for class_name, label in class_dirs.items():
        class_dir_path = os.path.join(data_root, class_name)
        if not os.path.isdir(class_dir_path):
             print(f"Uyarı: Sınıf dizini bulunamadı: {class_dir_path}")
             continue
        all_files = os.listdir(class_dir_path)
        image_files = [f for f in all_files if os.path.splitext(f)[1].lower() in img_extensions]

        if not image_files:
             print(f"Uyarı: {class_dir_path} dizininde görüntü dosyası bulunamadı.")
             continue

        for filename in tqdm(image_files, desc=f"İşleniyor {class_name}"):
            file_path = os.path.join(class_dir_path, filename)
            img_bgr = cv2.imread(file_path)
            if img_bgr is None:
                total_images_skipped += 1
                continue
            img_resized = cv2.resize(img_bgr, target_size, interpolation=cv2.INTER_LINEAR)
            if normalize: img_resized = img_resized.astype(np.float32) / 255.0
            img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
            processed_data['gray'].append(img_gray)
            if 'bgr' in color_spaces:
                processed_data['bgr'].append(img_resized)
            if 'hsv' in color_spaces:
                if normalize: 
                    img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
                else: 
                    img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
                processed_data['hsv'].append(img_hsv)
            if 'ycbcr' in color_spaces:
                if normalize: 
                    img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
                else: 
                    img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
                processed_data['ycbcr'].append(img_ycbcr)

            labels.append(label)
            total_images_processed += 1

    print(f"\nToplam işlenen görüntü: {total_images_processed}, atlanan: {total_images_skipped}")

    output_dtype = np.float32 if normalize else np.uint8
    numpy_data = {}
    for cs, data_list in processed_data.items():
        if data_list:
            try:
                numpy_data[cs] = np.array(data_list, dtype=output_dtype)
            except ValueError as e:
                print(f"Hata: {cs} verisi için numpy dizisi oluşturulurken şekil hatası: {e}")
                numpy_data[cs] = np.array([], dtype=output_dtype) 
        else:
            if cs == 'gray':
                numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0])
            else:
                numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0], 3)

    numpy_data['labels'] = np.array(labels, dtype=np.int32)
    return numpy_data


def load_prep_dfire(split_root_path, target_size, fire_class_ids,
    color_spaces=['bgr', 'hsv', 'ycbcr'], normalize=1):
    processed_data = {cs: [] for cs in color_spaces + ['gray']}
    labels = []
    img_extensions = ['.jpg', '.jpeg', '.png']
    annotation_extension = '.txt'
    images_dir = os.path.join(split_root_path, 'images')
    labels_dir = os.path.join(split_root_path, 'labels')

    if not os.path.isdir(images_dir):
        print(f"Hata: Resim dizini bulunamadı: {images_dir}")
        return None
    if not os.path.isdir(labels_dir):
        print(f"Hata: Etiket dizini bulunamadı: {labels_dir}")
        
        print("Uyarı: Etiket dizini bulunamadı, tüm görüntüler yangınsız (0) olarak etiketlenecektir.")
        labels_dir = None 

    all_image_files = []
    if os.path.isdir(images_dir):
        all_image_files = [f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in img_extensions]

    if not all_image_files:
         print(f"Uyarı: {images_dir} dizininde görüntü dosyası bulunamadı.")
         return None

    total_images_processed = 0
    total_images_skipped_read = 0
    total_images_skipped_annotation = 0 
    images_with_fire = 0
    images_without_fire = 0

    print(f"{os.path.basename(split_root_path)} bölünmesindeki {len(all_image_files)} görüntü işleniyor...")
    for filename in tqdm(all_image_files, desc=f"İşleniyor {os.path.basename(split_root_path)}"):
        image_path = os.path.join(images_dir, filename)
        image_name_without_ext = os.path.splitext(filename)[0]
        annotation_path = os.path.join(labels_dir, image_name_without_ext + annotation_extension) if labels_dir else None

        has_fire = False
        if annotation_path and os.path.exists(annotation_path):
            try:
                with open(annotation_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts and len(parts) > 0:
                            try:
                                class_id = int(parts[0])
                                if class_id in fire_class_ids:
                                    has_fire = True
                                    break 
                            except ValueError: pass 
            except Exception as e:
                total_images_skipped_annotation += 1
                has_fire = False 
        elif annotation_path and not os.path.exists(annotation_path):
            total_images_skipped_annotation += 1
            has_fire = False 

        label = 1 if has_fire else 0

        img_bgr = cv2.imread(image_path)
        if img_bgr is None:
            total_images_skipped_read += 1
            continue

        img_resized = cv2.resize(img_bgr, target_size, interpolation=cv2.INTER_LINEAR)

        if normalize:
             img_resized = img_resized.astype(np.float32) / 255.0
        
        img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        processed_data['gray'].append(img_gray)

        if 'bgr' in color_spaces:
             processed_data['bgr'].append(img_resized) 
        if 'hsv' in color_spaces:
            if normalize: 
                img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            else: 
                img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            processed_data['hsv'].append(img_hsv)
        if 'ycbcr' in color_spaces:
            
            if normalize: 
                img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
            else: 
                img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
            processed_data['ycbcr'].append(img_ycbcr)

        labels.append(label)
        total_images_processed += 1
        if has_fire:
            images_with_fire += 1
        else:
            images_without_fire += 1

    print(f"\nToplam işlenen görüntü: {total_images_processed}")
    print(f"Görüntü okuma hatası nedeniyle atlanan: {total_images_skipped_read}")
    print(f"Etiket işleme hatası/yokluğu nedeniyle atlanan: {total_images_skipped_annotation}")
    print(f"Yangın var (etiket 1): {images_with_fire}, Yangın yok (etiket 0): {images_without_fire}")

    output_dtype = np.float32 if normalize else np.uint8
    numpy_data = {}
    for cs, data_list in processed_data.items():
        if data_list:
             try:
                numpy_data[cs] = np.array(data_list, dtype=output_dtype)
             except ValueError as e:
                print(f"Hata: {cs} verisi için numpy dizisi oluşturulurken şekil hatası: {e}")
                numpy_data[cs] = np.array([], dtype=output_dtype) 
        else:
             if cs == 'gray':
                numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0])
             else: 
                numpy_data[cs] = np.array([], dtype=output_dtype).reshape(0, target_size[1], target_size[0], 3)

    numpy_data['labels'] = np.array(labels, dtype=np.int32)

    if numpy_data['labels'].shape[0] != total_images_processed:
        print(f"Uyarı: Etiket sayısı işlenen görüntü sayısıyla eşleşmiyor ({numpy_data['labels'].shape[0]} vs {total_images_processed}).")

    return numpy_data


def load_and_extract_features_memory_safe(config, feature_params):
    dataset_choice = config.get('dataset_choice', 'dfire')
    data_root = config.get('data_root')
    target_size = config.get('target_img_size')
    color_spaces_to_load = config.get('color_spaces_to_load', ['bgr', 'hsv', 'ycbcr'])
    normalize_pixels = config.get('normalize_pixels', 1)
    fire_class_ids = config.get('fire_class_ids', [])

    if not data_root or not target_size:
        print("Config 'data_root' or 'target_img_size' is missing.")
        return np.array([]), np.array([])

    image_label_pairs = []
    img_extensions = ['.jpg', '.jpeg', '.png']
    annotation_extension = '.txt'

    images_dir = os.path.join(data_root, 'images')
    labels_dir = os.path.join(data_root, 'labels')
    if not os.path.isdir(images_dir):
        print(f"Images directory not found: {images_dir}")
        return np.array([]), np.array([])
    if not os.path.isdir(labels_dir):
        print(f"Labels directory not found: {labels_dir}")
        return np.array([]), np.array([])

    all_image_files = [f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in img_extensions]
    if not all_image_files:
         print(f"No image files found in {images_dir}")
         return np.array([]), np.array([])

    print(f"Scanning {len(all_image_files)} images in {images_dir}...")
    for filename in tqdm(all_image_files, desc="Determining Labels"):
        image_path = os.path.join(images_dir, filename)
        image_name_without_ext = os.path.splitext(filename)[0]
        annotation_path = os.path.join(labels_dir, image_name_without_ext + annotation_extension)
        label = 1 if is_dfire_image_fire(annotation_path, fire_class_ids) else 0
        image_label_pairs.append((image_path, label))

    if not image_label_pairs:
        print("No image/label pairs found.")
        return np.array([]), np.array([])

    all_features_list = []
    all_labels_list = []
    total_images_processed = 0
    total_images_skipped_reading = 0
    total_images_skipped_feature = 0

    print(f"Processing {len(image_label_pairs)} images and extracting features...")
    for image_path, label in tqdm(image_label_pairs, desc="dfire memsafe feat exc."):
        img_bgr = cv2.imread(image_path)
        if img_bgr is None:
            total_images_skipped_reading += 1
            continue
        img_resized = img_bgr
        if target_size:
            img_resized = cv2.resize(img_bgr, target_size, interpolation=cv2.INTER_LINEAR)
        if normalize_pixels:
            img_resized = img_resized.astype(np.float32) / 255.0
    
        img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        img_dict_single = {'gray': img_gray}
        if 'bgr' in color_spaces_to_load:
            img_dict_single['bgr'] = img_resized
        if 'hsv' in color_spaces_to_load:
            if normalize_pixels:
                img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            else:
                img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            img_dict_single['hsv'] = img_hsv

        if 'ycbcr' in color_spaces_to_load:
             
             if normalize_pixels:
                img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
             else:
                 
                 img_ycbcr = cv2.cvtColor(img_resized, cv2.COLOR_BGR2YCrCb)
             img_dict_single['ycbcr'] = img_ycbcr

        features_single = combine_features(img_dict_single, feature_params)

        if features_single.size > 0:
            all_features_list.append(features_single)
            all_labels_list.append(label)
            total_images_processed += 1
        else:
            total_images_skipped_feature += 1

    print(f"Total images processed for features: {total_images_processed}")
    if total_images_skipped_reading > 0:
        print(f"Total images skipped due to read error: {total_images_skipped_reading}")
    if total_images_skipped_feature > 0:
        print(f"Total images skipped due to feature extraction error/empty features: {total_images_skipped_feature}")


    if not all_features_list:
        print("No features extracted from any image.")
        return np.array([]), np.array([])

    try:
        features_array = np.vstack(all_features_list).astype(np.float32)
    except ValueError as e:
        print(f"Error stacking features: {e}. Ensure all feature vectors have the same size.")
        return np.array([]), np.array([])
    labels_array = np.array(all_labels_list, dtype=np.int32)
    print(f"Successfully extracted features for {features_array.shape[0]} images.")
    return features_array, labels_array

base_dir = '..'
MODELS_DIR = os.path.join(base_dir, 'models')

TRANSFORMERS_DIR = MODELS_DIR

SCALER_FILENAME = 'scaler.pkl' 
PCA_FILENAME = 'pca_model.pkl' 
SCALER_PATH = os.path.join(TRANSFORMERS_DIR, SCALER_FILENAME)
PCA_PATH = os.path.join(TRANSFORMERS_DIR, PCA_FILENAME)

print("--- Test Verisi Yükleme ve Ön İşleme ---")
config_test_dfire = get_config('dfire')
feature_params = get_feature_params()
processed_test_data_dict = load_and_preprocess_data(config_test_dfire)
X_test_raw_features = np.array([])
y_test = np.array([])

if processed_test_data_dict:
    X_test_raw_features, y_test = extract_features(processed_test_data_dict, feature_params)
    if X_test_raw_features.shape[0] == 0:
        print("\nÖzellik çıkarma başarısız oldu veya test seti boş.")
else:
    print("\nTest verisi yükleme veya ön işleme başarısız oldu.")

X_test_processed = X_test_raw_features 

loaded_scaler = None
if os.path.exists(SCALER_PATH):
    try:
        print(f"\nScaler yükleniyor: {SCALER_PATH}")
        loaded_scaler = joblib.load(SCALER_PATH)
        print("Test özellikleri ölçeklendiriliyor...")
        X_test_processed = scale_features_transform(X_test_raw_features, loaded_scaler)
        if X_test_processed is None:
            print("Ölçeklendirme sonrası veri boş.")
            exit()
    except Exception as e:
        print(f"Hata: Scaler yüklenirken veya uygulanırken hata oluştu: {e}")
        loaded_scaler = None 
        X_test_processed = X_test_raw_features 
else:
    print(f"\nScaler dosyası bulunamadı: {SCALER_PATH}. Özellikler ölçeklendirilmeyecek.")

loaded_pca = None
if os.path.exists(PCA_PATH):
    try:
        print(f"\nPCA modeli yükleniyor: {PCA_PATH}")
        loaded_pca = joblib.load(PCA_PATH)
        print("Özelliklere PCA uygulanıyor...")
        X_test_processed = apply_pca_transform(X_test_processed, loaded_pca) 
        if X_test_processed is None:
             print("PCA sonrası veri boş.")
             exit()
    except Exception as e:
        print(f"Hata: PCA modeli yüklenirken veya uygulanırken hata oluştu: {e}")
        loaded_pca = None 
        
else:
    print(f"\nPCA dosyası bulunamadı: {PCA_PATH}. Boyut indirgeme uygulanmayacak.")

X_test_final = X_test_processed
if X_test_final.shape[0] == 0 or y_test.shape[0] == 0 or X_test_final.shape[0] != y_test.shape[0]:
    print("\nTest için hazırlanan özellik veya etiket verisinde sorun var. Testler başlatılamıyor.")
    print(f"X_test_final shape: {X_test_final.shape}, y_test shape: {y_test.shape}")
    exit()
print("\n--- Modeller Test Ediliyor ---")
if not os.path.exists(MODELS_DIR):
    exit()
model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith('.pkl')]
if not model_files:
    print(f"'{MODELS_DIR}' dizininde test edilecek .pkl dosyası bulunamadı.")
else:
    print(f"'{MODELS_DIR}' dizininde bulunan modeller: {len(model_files)} adet")
    for model_file in model_files:
        print(f"\n--- Model: {model_file} ---")
        model_path = os.path.join(MODELS_DIR, model_file)
        try:
            start_time = time.time()
            model = joblib.load(model_path)
            load_time = time.time() - start_time
            print(f"Model başarıyla yüklendi ({load_time:.4f} s).")
            print("Test veri seti üzerinde tahminler yapılıyor...")
            start_time = time.time()
            predictions = model.predict(X_test_final)
            predict_time = time.time() - start_time
            print(f"Tahminler tamamlandı ({predict_time:.4f} s).")
            print("\nSınıflandırma Raporu:")
            print(classification_report(y_test, predictions))
        except Exception as e:
            exit()
print("\n--- Tüm Model Testleri Tamamlandı ---")